In [ ]:
import json
import numpy as np
from skimage.draw import polygon
from skimage import io
from matplotlib import pyplot as plt

In [ ]:
import gzip
import json
#filename = '/home/garner1/tools/QuPath-0.2.0-Linux/QuPath-0.2.0/bin/test/polygons.json.gz'
filename = '/home/garner1/tools/QuPath-0.2.0-Linux/QuPath-0.2.0/bin/test/export/GTEX-1128S-0726.poly.json.gz'
with gzip.GzipFile(filename, 'r') as fin:
    data = json.loads(fin.read().decode('utf-8'))

In [ ]:
# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
x_list = []
y_list = []
count = 0
for entry in data:
    #print(count); count += 1
    poly = np.array(entry['geometry']['coordinates'][0]) # load coord of polygon
    mean = poly.mean(axis=0) 
    poly -= mean # subtract the mean
    mask = np.zeros(tuple(np.ceil(np.max(poly,axis=0) - np.min(poly,axis=0)).astype(int))).astype(int) # build an empty mask spanning the support of the polygon
    mini = np.min(poly,axis=0)
    poly -= mini # subtract the min to translate the mask 
    rr, cc = polygon(poly[:, 0], poly[:, 1], mask.shape) # get the nonzero mask locations
    mask[rr, cc] = 1
    rr = rr.astype(float);cc = cc.astype(float)
    # rescale back to original coordinates
    rr += mini[0]; cc += mini[1]
    rr += mean[0]; cc += mean[1]
    # update the list of nonzero entries
    x_list.extend(list(rr)) 
    y_list.extend(list(cc))

In [ ]:
del data

In [ ]:
xx = np.array(x_list).reshape((len(x_list),1))
yy = np.array(y_list).reshape((len(y_list),1))
arr = np.hstack((xx,yy))
#print(arr)
arr -= np.mean(arr,axis=0)
#print(arr)
#mask = np.zeros(tuple(np.ceil(np.max(arr,axis=0) - np.min(arr,axis=0)).astype(int))).astype(int) # build an empty mask spanning the support of the polygon
mini = np.min(arr,axis=0)
arr -= mini # subtract the min to translate the mask 
#print(arr)
#mask[arr[:,0].astype(int), arr[:,1].astype(int)] = 1

In [ ]:
from scipy.sparse import coo_matrix
row = np.rint(arr[:,0]).astype(int)
col = np.rint(arr[:,1]).astype(int)
mtx = coo_matrix((np.ones(row.shape), (row, col)), dtype=bool)

In [ ]:
r0 = 0
c0 = 0
select_row = np.arange(r0,r0+np.max(mtx.nonzero(),axis=1)[0])
select_col = np.arange(c0,c0+np.max(mtx.nonzero(),axis=1)[1])
fov = mtx.tocsr()[select_row,:][:,select_col]

plt.figure(figsize=(10,10))
io.imshow(fov.todense(),cmap='gray')
plt.show()